<a href="https://colab.research.google.com/github/Amine-OMRI/tweet-sentiment-extraction-kaggle-compete-1st-place-detailed-solution/blob/main/roberta_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

La solution a été implémentée en utilisant une version antérieure des bibliothèques 

1.   tokenizers==0.7.0
2.   transformers==2.9.0



In [ ]:
!pip install tokenizers==0.7.0

In [ ]:
!pip install transformers==2.9.0

In [ ]:
!pip install torchcontrib

[torchcontrib](https://github.com/pytorch/contrib) librairie contient des implémentations d'idées issues de récents articles sur l'apprentissage machine, ici elle a été utilisée pour réaliser #Stochastic Weight Averaging (SWA) Que je vous présenteai ultérieurement dans le Notebook.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Pour Commencer
**Ce notebook représente l'un des modèles de premier niveau réalisé par [Heartkilla](https://www.kaggle.com/aruchomu) : Artsem Zhyvalkouski le code source est sous `src/1st_level/roberta_base/` sur le [repo git](https://github.com/heartkilla/kaggle_tweet/tree/master/src/1st_level/roberta_base) où ils ont mis le code de leur solution**

---
Pour faire tourner la solution de la compétition, j'ai chargé quelques fichiers sur mon Gdrive qui sont : 


*   Les fichiers de données :</br>
    TRAINING_FILE = /content/drive/MyDrive/very_final/data/**train_folds.csv**</br>
    TEST_FILE = /content/drive/MyDrive/very_final/data/**test.csv**</br>
 

*   Les fichiers de configuration du tokeniser :</br>

    /content/drive/MyDrive/very_final/roberta_tokenizer/**vocab.json**</br>
    /content/drive/MyDrive/very_final/roberta_tokenizer/**merges.txt**</br>


J'ai également transféré les fichiers de **code** directement sur **Colab** en tant que fichiers d'entrée afin de pouvoir les importer et les utiliser, en même temps que j'ai copié leur code afin de pouvoir le commenter et remplir l'objectif de travail demandé.
* config.py
* dataset.py
* engine.py
* evaluate.py
* infer.py
* models.py
* train.py
* utils.py

#config.py

---

Le fichier **config.py** (configuration) contient les paramètres et les réglages initiaux des modules et des méthodes de la solution. L'importation du fichier config.py permet d'utiliser les variables et les fonctions du fichier config.py dans les autres fichiers de la solution.

In [6]:
import tokenizers

# Paths

TOKENIZER_PATH = '/content/drive/MyDrive/very_final/roberta_tokenizer'
TRAINING_FILE = '/content/drive/MyDrive/very_final/data/train_folds.csv'
TEST_FILE = '/content/drive/MyDrive/very_final/data/test.csv'
SUB_FILE = '/content/drive/MyDrive/very_final/data/sample_submission.csv'
MODEL_SAVE_PATH = '/content/drive/MyDrive/very_final/roberta_base/model_save'
TRAINED_MODEL_PATH = '/content/drive/MyDrive/very_final/roberta_base/model_save'

# Model config
##    Le tout premier modèle qui a été utilisé dans les modèles de 1er niveau
##    est roberta-base pour le modèle d'AQ par deepset.ai (Squad pretrained 
##    weights)) pré-entrainé sur la base SQuAD 2.0
MODEL_CONFIG = 'deepset/roberta-base-squad2'
#---------------------------------------------------------------------------------


# Model params
# Global Seed to initialize the pseudo-random number generator
# Pour assurer d'avoir les memes resultats d'une lancée a une autre
SEED = 25
# Nombre des folds pour l'entrainement par fold
N_FOLDS = 5
# Nombre d'EPOCHS de lentrainment des modéles 
EPOCHS = 4
LEARNING_RATE = 4e-5
PATIENCE = None
EARLY_STOPPING_DELTA = None
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
MAX_LEN = 96  # actually = 86

## Expliqué ci-dessous
TOKENIZER = tokenizers.ByteLevelBPETokenizer(
    vocab_file=f'{TOKENIZER_PATH}/vocab.json',
    merges_file=f'{TOKENIZER_PATH}/merges.txt',
    lowercase=True,
    add_prefix_space=True)
# 768 est la dimension des Embeddings 
HIDDEN_SIZE = 768
N_LAST_HIDDEN = 12
HIGH_DROPOUT = 0.5
SOFT_ALPHA = 0.6
WARMUP_RATIO = 0.25
WEIGHT_DECAY = 0.001
#Stochastic Weight Averaging (SWA) :
# les paramétres de l'optimiser 
USE_SWA = False
SWA_RATIO = 0.9
SWA_FREQ = 30

Source : **huggingface tokenizers**
Fournit une implémentation des tokenizers les plus utilisés aujourd'hui, en mettant l'accent sur les performances et la polyvalence.
Utilisé pour :
* Entraîner de nouveaux vocabulaires et tokeniser à l'aide de 4 tokenizers pre-made (Bert WordPiece et les 3 versions les plus courantes de BPE).
* Extrêmement rapide (tant pour l'entraînement que pour la tokenisation), grâce à l'implémentation de Rust. Il le faut moins de 20 secondes pour convertir un Go de texte en tokens sur le processeur d'un serveur.
* Facile à utiliser, mais aussi extrêmement polyvalent.
* Conçu pour la recherche et la production.
* La normalisation s'accompagne d'un suivi des alignements. Il est toujours possible d'obtenir la partie de la phrase originale qui correspond à un jeton donné.
* Effectue tout le prétraitement : Tronquer, Pad, ajouter les tokens spéciaux dont votre modèle a besoin.





Le tokenizer de RoBERTa est basé sur le tokenizer GPT-2, les fichiers vocab/merges sont constitués lors de l'entrainement du BBPE [(Byte-level Byte-Pair-Encoding)](https://arxiv.org/pdf/1909.03341.pdf) et utilisés pour encoder les sentences, le tokenizer tokenize d'abord en se basant sur le fichier **merges.txt**.</br>

Voici un exemple :</br>

```
['What', "'s", 'Ġup', 'Ġwith', 'Ġthe', 'Ġtoken', 'izer', '?']</br>

```
le caractère ```Ġ``` signifie un espace</br>


Et ensuite, selon les valeurs dans le fichier **vocab.json**, ces tokens sont alors remplacés par leurs indices :</br>
```
[   'What',     "'s",    'Ġup',  'Ġwith',   'Ġthe', 'Ġtoken',   'izer',      '?']
---- becomes ----
[     2061,      338,      510,      351,      262,    11241,     7509,       30]
```

#dataset.py

---
le fichier dataset.py pour assurer que toutes les données des tweets soient stockées au même endroit et soient utilisées pour charger les données avec le **dataloader** par la suite.

* Il contient également une implémentation d'un  Map-style datasets (the **TweetDataset** class)  qui implémente les méthodes **__getitem__()** et **__len__()** et qui représente toutes les propriétés des ids des tweets, les offsets, orig_start/orig_end, start_labels/end_labels, mask, token_type_ids, ...

* Egalement une implémentation de la méthode **process_data** qui traitera qui calculera toutes ces propriétés et les extraira des tweets, selected_text, sentiment en utilisant le tokeniser crée dans le fichier config.py.


Voyons voir a quoi ressemblent les données d'entrainement

In [7]:
import config
import pandas as pd
df_train = pd.read_csv(config.TRAINING_FILE)
df_train.head()

,textID,text,selected_text,sentiment,kfold
0,d0c214ad3a,good mornig to everone... it`s a great morning...,good mornig to everone... it`s a great morning...,positive,0
1,7d093817af,LOL. You know me. I aim to please.,I aim to please.,positive,0
2,21eacf7e58,Was at Ruby Skye last night as well! Superb s...,Superb set by Steve.,positive,0
3,d0f94d66ab,does not like ups much today...,does not like,negative,0
4,a025e21634,Nothing like In `n` Out and a LOST marathon af...,long day of work.,negative,0


**===>  `selected_text` est la valeur à prédire**

In [9]:
df_train.shape

(27480, 5)

In [10]:
tweet = df_train.text.values[3]
selected_text = df_train.selected_text[3]
tweet = ' ' + ' '.join(str(tweet).split())
selected_text = ' ' + ' '.join(str(selected_text).split())
tweet

' does not like ups much today...'

In [ ]:
selected_text 

' does not like'

In [ ]:
len_sel_text = len(selected_text) - 1
## récpèrerl'idice de début et de fin de selected_text
idx_0 = None
idx_1 = None
## i : indice du caractére, e c'est le caractére SI e est le caractére 
## d'indice 1 (just apré lespace quona rajouté)
for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
  if ' ' + tweet[ind:ind + len_sel_text] == selected_text:
    idx_0 = ind
    idx_1 = ind + len_sel_text - 1
    break
print('l\'indexe de début:',idx_0,'l\'indexe de fin:', idx_1, 'du text séléctionné qui représente le target')

l'indexe de début: 1 l'indexe de fin: 13 du text séléctionné qui représente le target


Assigner `1` à chaque caractère du tweet s'il fait partie du selected_text sinon `0`, comme dans l'exemple ci-dessus. 

In [ ]:
# Assign 1 as target for each char in sel_text
char_targets = [0] * len(tweet)
if idx_0 is not None and idx_1 is not None:
  for ct in range(idx_0, idx_1 + 1):
    char_targets[ct] = 1
char_targets

[0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
tweet

' does not like ups much today...'

In [ ]:
tokenized_tweet = TOKENIZER.encode(tweet)
input_ids_original = tokenized_tweet.ids
input_ids_original

[473, 45, 101, 12744, 203, 452, 734]

Une **nouvelle méthode** pour les tokenizers : **tokenize_with_offsets**. En plus de renvoyer les tokens, elle renvoie les intervalles dans le texte original auxquels les tokens correspondent, cette méthode nous permet de récupérer le token ou le mot qui correspond a un **id** donné.

In [ ]:
tweet_offsets = tokenized_tweet.offsets
tweet_offsets

[(0, 5), (5, 9), (9, 14), (14, 18), (18, 23), (23, 29), (29, 32)]

Récupérer le texte (les tokens) original en se basant sur les offsets (leur intervalle) : 

In [ ]:
tweet[tweet_offsets[0][0]:tweet_offsets[0][1]] + tweet[tweet_offsets[1][0]:tweet_offsets[1][1]]

' does not'

Ce code récupère les **target_ids** qui sont les id des tokens cibles qui représentent le texte sélectionné (chaque caractère est représenté (codé) par le chiffre `"1"`)

In [ ]:
target_ids = []
for i, (offset_0, offset_1) in enumerate(tweet_offsets):
  print(i,'offset (',offset_0,',', offset_1,')')
  print(char_targets[offset_0:offset_1])
  print(sum(char_targets[offset_0:offset_1]))
  if sum(char_targets[offset_0:offset_1]) > 0:
    target_ids.append(i)
print('target_ids : ',target_ids)

0 offset ( 0 , 5 )
[0, 1, 1, 1, 1]
4
1 offset ( 5 , 9 )
[1, 1, 1, 1]
4
2 offset ( 9 , 14 )
[1, 1, 1, 1, 1]
5
3 offset ( 14 , 18 )
[0, 0, 0, 0]
0
4 offset ( 18 , 23 )
[0, 0, 0, 0, 0]
0
5 offset ( 23 , 29 )
[0, 0, 0, 0, 0, 0]
0
6 offset ( 29 , 32 )
[0, 0, 0]
0
target_ids :  [0, 1, 2]


* **Soft Jaccard labels :**</br>
Custom loss Jaccard-based Soft Labels: Étant donné que la Cross Entropy n'optimise pas directement l'indice de Jaccard, Heartkilla a essayé différentes fonctions de Loss pour pénaliser davantage les prédictions lointaines que les prédictions proches, il a donc trouvé une Loss personnalisée en calculant l'indice de Jaccard au niveau du token. Il a ensuite utilisé ces nouveaux labels cibles et a optimisé la divergence.</br> 
Alpha c'est un paramètre permettant d'équilibrer l'étiquetage habituel basé sur la Cross Entropy et l'indice de Jaccard </br>
<img src = 'https://camo.githubusercontent.com/3925753ce615ec71960dad457401aedefc7b611a2b11a3cb86eb060772dce880/68747470733a2f2f7777772e676f6f676c65617069732e636f6d2f646f776e6c6f61642f73746f726167652f76312f622f6b6167676c652d757365722d636f6e74656e742f6f2f696e626f7825324632303030353435253246393334316265646532383236336263663065396262323539616337393033333825324653637265656e25323053686f74253230323032302d30352d3330253230617425323031372e33312e32322e706e673f67656e65726174696f6e3d3135393234303530323835353638343226616c743d6d65646961'></br>


In [ ]:
## La mesure d'évaluation qui a été mentionnée sur le présent de la competition 
def jaccard_array(a, b):answer
    """Calculates Jaccard on arrays."""
    a = set(a)
    b = set(b)
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
import numpy as np
targets_start = target_ids[0]
targets_end = target_ids[-1]
n = len(input_ids_original)
## id des tokens dans le tweet
sentence = np.arange(n)
## id  des tokens qui forment le label (le selected_text)
answer = sentence[targets_start:targets_end + 1]

In [ ]:
sentence

array([0, 1, 2, 3, 4, 5, 6])

Le tableaux retourné a ce niveau représente les indices du target (text séléctionné) dans la sentence orginal.

In [ ]:
answer

array([0, 1, 2])

C'est bien la target selected_text: '` does not like` '

In [ ]:
sentence[targets_start:targets_end + 1]

array([0, 1, 2])

In [ ]:
for i in range(targets_end + 1):
  ## calculate the jaccard indexe
  ## answer = array([0, 1, 2]) qui est les indice du atrget selected_text: 'does not like'
  jac = jaccard_array(answer, sentence[i:targets_end + 1])

jaccard indexe du token (0,2) =1.0
jaccard indexe du token (1,2) =0.6666666666666666
jaccard indexe du token (2,2) =0.3333333333333333


jaccard indexe du token (0,2) =1.0 puisque le texte correspondant à l'index de **(0)** jusqu'à la target_end **(2)** dans la phrase originale (`' does not like ups much today...'`) est exactement le taget qui est `'does not like'`.

In [ ]:
start_labels = np.zeros(n)
for i in range(targets_end + 1):
    ## calculate the jaccard indexe 
    jac = jaccard_array(answer, sentence[i:targets_end + 1])
    print('jaccard indexe du token ('+ str(i) +','+ str(targets_end)+') ='+ str(jac))
    start_labels[i] = jac + jac ** 2
 
## Alpha est un paramètre d'équilibre entre la CE et le Jaccard-based labeling
start_labels = (1 - config.SOFT_ALPHA) * start_labels / start_labels.sum()
start_labels[targets_start] += config.SOFT_ALPHA
start_labels

jaccard indexe du token (0,2) =1.0
jaccard indexe du token (1,2) =0.6666666666666666
jaccard indexe du token (2,2) =0.3333333333333333


array([0.825, 0.125, 0.05 , 0.   , 0.   , 0.   , 0.   ])

In [ ]:
import numpy as np
import torch

import config

## La mesure d'évaluation qui a été mentionnée sur le présent de la competition 
def jaccard_array(a, b):
    """Calculates Jaccard on arrays."""
    a = set(a)
    b = set(b)
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


def process_data(tweet, selected_text, sentiment,
                 tokenizer, max_len):
    """Preprocesses one data sample and returns a dict
    with targets and other useful info.
    """
    ## Pour un tweet donné:
    ## récpèrer le text des tweet sous forme d'une str séparée par espace (commence par ' ' déja)
    tweet = ' ' + ' '.join(str(tweet).split())
    ## récpèrer le text des selected_text
    selected_text = ' ' + ' '.join(str(selected_text).split())

    ## récpèrer le len de selected_text (-1 puisque python commence à partir de 0)
    len_sel_text = len(selected_text) - 1

    ## récpèrer l'idice de début et de fin de selected_text
    idx_0 = None
    idx_1 = None
    ## i : indice du caractére, e c'est le caractére SI e est le caractére 
    ## d'indice 1 (just apré lespace qu'ona rajouté au début de selected_text)
    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        ## récupérer l'idx de début et fin du selected_text dans le text du tweet
        if ' ' + tweet[ind:ind + len_sel_text] == selected_text:
            idx_0 = ind
            idx_1 = ind + len_sel_text - 1
            break

    ## Assignez 1 à chaque caractère du tweet s'il fait partie du selected_text
    ## sinon 0, comme dans l'exemple ci-dessus.    
    char_targets = [0] * len(tweet)
    if idx_0 is not None and idx_1 is not None:
        for ct in range(idx_0, idx_1 + 1):
            char_targets[ct] = 1

    ## tokeniser le texte du tweet en utilisant le tokeniser que nous avons 
    ## créé sur le fichier config.py
    tokenized_tweet = tokenizer.encode(tweet)
    ## récupérer les indices affectés par le tokeniser à chaque jeton
    input_ids_original = tokenized_tweet.ids
    ## Cette methode ".offsets" permet de récupérer les intervalles dans le
    ## texte original auxquels les tokens correspondent.
    tweet_offsets = tokenized_tweet.offsets

    ## Ce code récupère les target_ids qui sont les id des tokens cibles qui
    ## représentent le texte sélectionné (chaque caractère est représenté par 1)
    target_ids = []
    for i, (offset_0, offset_1) in enumerate(tweet_offsets):
        if sum(char_targets[offset_0:offset_1]) > 0:
            target_ids.append(i)
    ## idx début du text du target
    targets_start = target_ids[0]
    ## idx fin du text du target
    targets_end = target_ids[-1]

    # Sentimadd_prefix_spaceent 'word' id in vocab
    ## Encoder le feature de sentiment
    sentiment_id = {'positive': 1313,
                    'negative': 2430,
                    'neutral': 7974}

    # Soft Jaccard labels
    #C'est la méthode d'étiquetage personnalisée qui a été adoptée par les compétiteurs. 
    # ----------------------------------
    n = len(input_ids_original)
    sentence = np.arange(n)
    answer = sentence[targets_start:targets_end + 1]
    start_labels = np.zeros(n)
    for i in range(targets_end + 1):
        jac = jaccard_array(answer, sentence[i:targets_end + 1])
        start_labels[i] = jac + jac ** 2
        
    ## Alpha est un paramètre d'équilibre entre l'étiquetage (labeling) Cross Enthropy 
    ## habituel et l'étiquetage basé sur la carte Jaccard (Jaccard-based labeling).
    start_labels = (1 - config.SOFT_ALPHA) * start_labels / start_labels.sum()
    start_labels[targets_start] += config.SOFT_ALPHA

    end_labels = np.zeros(n)
    for i in range(targets_start, n):
        jac = jaccard_array(answer, sentence[targets_start:i + 1])
        end_labels[i] = jac + jac ** 2
    end_labels = (1 - config.SOFT_ALPHA) * end_labels / end_labels.sum()
    end_labels[targets_end] += config.SOFT_ALPHA
    ## Les nouveaux labels qui seront utilisés pour améliorer et garantir que le modèle apprendra correctement
    start_labels = [0, 0, 0, 0] + list(start_labels) + [0]
    end_labels = [0, 0, 0, 0] + list(end_labels) + [0]
    # ----------------------------------

    ## l'input pour RoBERTa
    input_ids = [0] + [sentiment_id[sentiment]] + [2] + \
                [2] + input_ids_original + [2]
    ## Pas de types de token dans RoBERTa (tous a 0)
    token_type_ids = [0, 0, 0, 0] + [0] * (len(input_ids_original) + 1)
    ## Mask de l'input sans padding
    mask = [1] * len(token_type_ids)
    ## Identifiants des caractères de début et de fin pour chaque mot, y compris les nouveaux tokens
    tweet_offsets = [(0, 0)] * 4 + tweet_offsets + [(0, 0)]
    ## Ids des mots dans le tweet qui ont un caractère cible, y compris les nouveaux tokens
    targets_start += 4
    targets_end += 4
    orig_start = 4
    orig_end = len(input_ids_original) + 3

    ## Avant que RoBERTa puisse traiter ces données en entrée, nous devrons rendre 
    ## tous les vecteurs de même taille en ajoutant (padding ) des phrases plus courtes avec le token id 0. 
    ## Après le padding, nous avons une matrice / un tenseur 
    ## qui est prêt à être passé à RoBERTa.
    ## Input padding: new mask, token type ids, tweet offsets
    ## s'il y'en à du padding 
    padding_len = max_len - len(input_ids)
    if padding_len > 0:
      ## on récupère les input_ids, mask, token_type_ids, tweet_offsets, end_offsets
      input_ids = input_ids + ([1] * padding_len)
      mask = mask + ([0] * padding_len)
      token_type_ids = token_type_ids + ([0] * padding_len)
      tweet_offsets = tweet_offsets + ([(0, 0)] * padding_len)
      start_labels = start_labels + ([0] * padding_len)
      end_labels = end_labels + ([0] * padding_len)
    ## Compute le targets_select
    targets_select = [0] * len(token_type_ids)
    for i in range(len(targets_select)):
        if i in target_ids:
            targets_select[i + 4] = 1

    ## la sortie pour un tweet donné
    return {'ids': input_ids,
            'mask': mask,
            'token_type_ids': token_type_ids,
            'start_labels': start_labels,
            'end_labels': end_labels,
            'orig_start': orig_start,
            'orig_end': orig_end,
            'orig_tweet': tweet,
            'orig_selected': selected_text,
            'sentiment': sentiment,
            'offsets': tweet_offsets,
            'targets_select': targets_select}


## Une classe de pratique pour que toutes les données des tweets soient stockées 
## au même endroit et qui sera utilisé pour charger les donéées avec dataloader aprés

## Map-style datasets
## A map-style dataset is one that implements the __getitem__() and __len__() 
## protocols, and represents a map from (possibly non-integral) indices/keys to data samples.
class TweetDataset:
    '''
    définir un objet(classe) qui contient toutes les données des tweets et implémenter 
    la méthode (pre-buil) _len_  qui retourne le nombre de tweets et la méthode 
    __getitem__ qui traite les données des tweets et calcule toutes les  tenseur 
    (torch.tensor) qui sera alimenté par le modèle : 'ids', 'mask', 'token_type_ids', 
    'start_labels', 'end_labels', 'orig_start', 'orig_end', 'orig_tweet', 
    'orig_selected', 'sentiment', 'offsets', 'targets_select'.
    '''
    def __init__(self, tweets, sentiments, selected_texts):
        self.tweets = tweets
        self.sentiments = sentiments
        self.selected_texts = selected_texts
        self.max_len = config.MAX_LEN
        self.tokenizer = config.TOKENIZER

    def __len__(self):
        return len(self.tweets)

    def __getitem__(self, item):
        """Returns preprocessed data sample as dict with
        data converted to tensors.
        """
        data = process_data(self.tweets[item],
                            self.selected_texts[item],
                            self.sentiments[item],
                            self.tokenizer,
                            self.max_len)

        return {'ids': torch.tensor(data['ids'], dtype=torch.long),
                'mask': torch.tensor(data['mask'], dtype=torch.long),
                'token_type_ids': torch.tensor(data['token_type_ids'],
                                               dtype=torch.long),
                'start_labels': torch.tensor(data['start_labels'],
                                             dtype=torch.float),
                'end_labels': torch.tensor(data['end_labels'],
                                           dtype=torch.float),
                'orig_start': data['orig_start'],
                'orig_end': data['orig_end'],
                'orig_tweet': data['orig_tweet'],
                'orig_selected': data['orig_selected'],
                'sentiment': data['sentiment'],
                'offsets': torch.tensor(data['offsets'], dtype=torch.long),
                'targets_select': torch.tensor(data['targets_select'],
                                               dtype=torch.float)}

deux techniques ont été utilisées afin d'assurer la même longueur des séquences d'entrée:
* **Le masquage (Masking):** est un moyen d'indiquer aux couches de traitement des séquences que certains blocs sont manquants dans une entrée et qu'ils doivent donc être ignorés lors du traitement des données.

* **Le padding** est une forme spéciale de masquage où les étapes masquées se trouvent au au début d'une séquence. Le remplissage (padding) vient de la nécessité de coder les données de la séquence en lots consécutifs (contiguous batches): afin que toutes les séquences d'un batch soient conformes à une longueur standard donnée, il est nécessaire de remplir ou de tronquer certaines séquences.

# engine.py

---
Le fichier engine.py contient une definition de la fonction loss ainsi que l'implementation des deux fonction:</br>
* train : qui permet de mettre le model en mode entrainment
* eval: qui permet de mettre le model en mode evaluation 



In [ ]:
import numpy as np
import torch
import tqdm

import utils

## Definition de la fonction loss
def loss_fn(start_logits, end_logits,
            start_positions, end_positions):
    ## Appliquer la fonction \log(\text{Softmax}(x))log(Softmax(x)) à une entrée 
    ## n-dimensionnelle Tenseur ici dim = 1
    m = torch.nn.LogSoftmax(dim=1)
    ## La mesure de la divergence de Kullback-Leibler est une mesure de distance
    ## utile pour les distributions continues  
    loss_fct = torch.nn.(KLDivLoss)
    ## calculer la loss par apport a la prédiction du caractére de début du target(selected_text)
    start_loss = loss_fct(m(start_logits), start_positions)
    ## calculer la loss par apport a la prédiction du caractére de fin du target
    end_loss = loss_fct(m(end_logits), end_positions)
    ## La valeur de loss totale et la somme des deux loss par apprt a la prédiction 
    ## caractére de debut et fin du target
    total_loss = (start_loss + end_loss)
    return total_loss

## définition de la fonction train
def train_fn(data_loader, model, optimizer, device, scheduler=None):
    ## model.train() permet de mettre le modèle en mode train (il calcule les gradients)
    model.train()
    ## Permet de stocke la valeur moyenne actuelle et applique deux méthodes :
    ## reset : qui remet toutes les valeurs à zéro 
    ## update : qui met à jour l'objet en y ajoutant de nouvelles valeurs, ici il s'agit de la valeur de la loss
    losses = utils.AverageMeter()
   
    ## tqdm nous permettre de créer une progressbar en fonction de la longueur des données
    tk0 = tqdm.tqdm(data_loader, total=len(data_loader))

    for bi, d in enumerate(tk0):
        ## recupérer l'id du tweet
        ids = d['ids']
        ## récupérer les ids des tokens
        token_type_ids = d['token_type_ids']
        ## récupérer le mask du tweet
        mask = d['mask']
        ## la valeur du start/end lable calculer en utilisant Jaccard-based labeling
        start_labels = d['start_labels']
        end_labels = d['end_labels']

        ## nn.Module.to function permet de déplacer le modèle\tensors vers le GPU
        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        start_labels = start_labels.to(device, dtype=torch.float)
        end_labels = end_labels.to(device, dtype=torch.float)

        ## mettre les gradients à zéro avant de commencer à faire de la backpropragation  
        model.zero_grad()

        ## applique un forward pass et récuperer l'output
        outputs_start, outputs_end = \
            model(ids=ids, mask=mask, token_type_ids=token_type_ids)
        ## Calculer la valeur de la loss
        loss = loss_fn(outputs_start, outputs_end,
                       start_labels, end_labels)
        ## Calculer les gradiants
        loss.backward()
        ## Ajuster les poids de notre modele
        optimizer.step()
        ## un programmateur de taux d'apprentissage basé sur le temps
        ## - il est contrôlé par le paramètre de décroissance(decay) de l'optimiser
        scheduler.step()
        ## mettre a jour la valeur sauvegardé de la loss 
        losses.update(loss.item(), ids.size(0))
        tk0.set_postfix(loss=losses.avg)

## définition de la fonction de l'évaluation
def eval_fn(data_loader, model, device):
    ## model.eval() met le modèle en mode évaluation (il calcule pas les gradients)
    model.eval()
    ## récupérer la valeur de la loss
    losses = utils.AverageMeter()
    ## récupérer la valeur de la métric de jaccards
    jaccards = utils.AverageMeter()
    ## Le wrapper "with torch.no_grad()" met temporairement tous les tensors avec require_grad à false
    with torch.no_grad():
        ## passer les donnnée d'évaluation avec une progressbar
        tk0 = tqdm.tqdm(data_loader, total=len(data_loader))
        for bi, d in enumerate(tk0):
            ids = d['ids']
            token_type_ids = d['token_type_ids']
            mask = d['mask']
            start_labels = d['start_labels']
            end_labels = d['end_labels']
            orig_start = d['orig_start']
            orig_end = d['orig_end']
            orig_selected = d['orig_selected']
            orig_tweet = d['orig_tweet']
            offsets = d['offsets']
            
            ## nn.Module.to function permet de déplacer le modèle\tensors vers le GPU
            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            start_labels = start_labels.to(device, dtype=torch.float)
            end_labels = end_labels.to(device, dtype=torch.float)

            outputs_start, outputs_end = \
                model(ids=ids, mask=mask, token_type_ids=token_type_ids)
            loss = loss_fn(outputs_start, outputs_end,
                           start_labels, end_labels)
            ## récupérer les outputs start/stop prédites
            outputs_start = outputs_start.cpu().detach().numpy()
            outputs_end = outputs_end.cpu().detach().numpy()
            ## lancé le calcul de lindices de jaccard qui permet d'evaluer le modele
            jaccard_scores = []
            for px, tweet in enumerate(orig_tweet):
                ## recupérer la valeur réelle du target
                selected_tweet = orig_selected[px]
                jaccard_score, _ = \
                    utils.calculate_jaccard(original_tweet=tweet,
                                            target_string=selected_tweet,
                                            start_logits=outputs_start[px, :],
                                            end_logits=outputs_end[px, :],
                                            orig_start=orig_start[px],
                                            orig_end=orig_end[px],
                                            offsets=offsets[px])
                jaccard_scores.append(jaccard_score)
            ## mettre a jour la valeur de jaccard
            jaccards.update(np.mean(jaccard_scores), ids.size(0))
            ## pareil pour la valeur de la loss
            losses.update(loss.item(), ids.size(0))
            tk0.set_postfix(loss=losses.avg, jaccard=jaccards.avg)

    print(f'Jaccard = {jaccards.avg}')

    return jaccards.avg

# evaluate.py

---
La même implémentation de la fonction d'évaluation qui a été implémentée dans le fichier **engin.py**, celle-ci peut être utilisée pour effectuer une évaluation directement en utilisant le fichier **evaluate.py**.**bold text**


In [ ]:
import torch
import numpy as np
import pandas as pd
import transformers
import tqdm.autonotebook as tqdm

import utils
import config
import models
import dataset

## définition de la fonction de l'évaluation
def eval_fn(data_loader, model, device):
    model.eval()
    ## récupérer la valeur de la métric de jaccards
    jaccards = utils.AverageMeter()
    ## comme j'avais commenter sur le code ci-dessous
    ## Le wrapper "with torch.no_grad()" met temporairement tous les tensors avec require_grad à false
    with torch.no_grad():
        tk0 = tqdm.tqdm(data_loader, total=len(data_loader))
        for bi, d in enumerate(tk0):
            ids = d['ids']
            token_type_ids = d['token_type_ids']
            mask = d['mask']
            start_labels = d['start_labels']
            end_labels = d['end_labels']
            orig_start = d['orig_start']
            orig_end = d['orig_end']
            orig_selected = d['orig_selected']
            orig_tweet = d['orig_tweet']
            offsets = d['offsets']
            ## déplacer le modèle\tensors vers le GPU/CPU
            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            start_labels = start_labels.to(device, dtype=torch.float)
            end_labels = end_labels.to(device, dtype=torch.float)

            outputs_start, outputs_end = \
                model(ids=ids, mask=mask, token_type_ids=token_type_ids)

            outputs_start = outputs_start.cpu().detach().numpy()
            outputs_end = outputs_end.cpu().detach().numpy()

            jaccard_scores = []
            for px, tweet in enumerate(orig_tweet):
                selected_tweet = orig_selected[px]
                jaccard_score, _ = \
                    utils.calculate_jaccard(original_tweet=tweet,
                                            target_string=selected_tweet,
                                            start_logits=outputs_start[px, :],
                                            end_logits=outputs_end[px, :],
                                            orig_start=orig_start[px],
                                            orig_end=orig_end[px],
                                            offsets=offsets[px])
                jaccard_scores.append(jaccard_score)

            jaccards.update(np.mean(jaccard_scores), ids.size(0))
            tk0.set_postfix(jaccard=jaccards.avg)

    return jaccards.avg


def run(fold):
    ## Lecture des données de l'entrainement
    dfx = pd.read_csv(config.TRAINING_FILE)

    ## Lecture des données de la validation
    df_valid = dfx[dfx.kfold == fold].reset_index(drop=True)
    ## Les types de tenseurs CUDA, qui implémentent la même fonction que les 
    ## tenseurs CPU, mais qui utilisent les GPU pour le calcul
    device = torch.device('cuda')
    ## le modèle hérite de la class PreTrainedModel
    ## c'est un modéle pré-entrainé sur Squad2
    ## le modéle précisé dans la class config c'est bien 'deepset/roberta-base-squad2' 
    model_config = transformers.RobertaConfig.from_pretrained(config.MODEL_CONFIG)
    ##  Pour assurer que le modèle rendre tous les hidden_state (weights).
    model_config.output_hidden_states = True
    
    ## Crée une instance de la classe TweetModel avec la config crée just avant
    model = models.TweetModel(conf=model_config)
    model.to(device)
    ## La fonction load_state_dict() prend un objet du dictionnaire, tet elle
    ## charge le state_dict sérialisé et sauvegardé du modèle
    model.load_state_dict(torch.load(
        f'{config.TRAINED_MODEL_PATH}/model_{fold}.bin'))
    
    ## model.eval() met le modèle en mode évaluation (il calcule pas les gradients)
    model.eval()

    ## Préparé les tweets de validation selon la methode dataset.TweetDataset() qui prépare toutes les données des tweets
    valid_dataset = dataset.TweetDataset(
        tweets=df_valid.text.values,
        sentiments=df_valid.sentiment.values,
        selected_texts=df_valid.selected_text.values)
    ## chargement des données de validation en utilisant dataLoader de Pytorch 
    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=config.VALID_BATCH_SIZE,
        num_workers=4,
        shuffle=False)
    ## Le fait de définir l'argument num_workers comme un nombre entier positif
    ## activera le chargement de données multiprocessus avec le nombre spécifié de processus de chargement des travailleurs
    ## calculer l'indice de jaccard
    jaccard = eval_fn(valid_data_loader, model, device)

    return jaccard

##  if __name__ == "main" ' bloc pour empêcher l'exécution de (certain) code lors
##  de l'importation du module. En bref, __name__ est une variable définie pour 
##  chaque script qui définit si le script est exécuté en tant que module principal 
##  ou s'il est exécuté en tant que module importé.
if __name__ == '__main__':
    utils.seed_everything(config.SEED)
    ## Lise qui va contenir le score de chaque folds
    fold_scores = []
    ## N_FOLDS est a 5
    for i in range(config.N_FOLDS):
        fold_score = run(i)
        fold_scores.append(fold_score)
    ## Afficher les score de chaque folds et le score moyen
    for i in range(config.N_FOLDS):
        print(f'Fold={i}, Jaccard = {fold_scores[i]}')
    print(f'Mean = {np.mean(fold_scores)}')
    print(f'Std = {np.std(fold_scores)}')


Fold=0, Jaccard = 0.7163026420895469
Fold=1, Jaccard = 0.7108215431021135
Fold=2, Jaccard = 0.7158955035127735
Fold=3, Jaccard = 0.7166221978997807
Fold=4, Jaccard = 0.7109265704332721
Mean = 0.7141136914074974
Std = 0.002655369837057321


**TORCH.UTILS.DATA**
Au cœur de l'utilitaire de chargement de données PyTorch se trouve la classe ```torch.utils.data.DataLoader```. Elle représente un Python itérable sur un ensemble de données, avec le support de:</br>
* map-style et iterable-style datasets,

* la personnalisation de l'ordre de chargement des données,

* le dosage automatique,

* chargement de données à un ou plusieurs processus,

* épinglage automatique de la mémoire.

# infer.py

---
Ce code représente l'implémentation d'une Forward passe pour prédire le texte sélectionné (start/end_labels) des tweets donné, on peut dire que cela effectue la même tâche que la méthode .predict() en ML puisque ya pas la notion de l'apprentissage c'est just une propagation dans le modele et récupération de l'output.


In [ ]:
import pickle

import pandas as pd
import torch
import transformers
import tqdm

import config
import models
import dataset
import utils


def run():
    df_test = pd.read_csv(config.TEST_FILE)
    df_test.loc[:, 'selected_text'] = df_test.text.values

    device = torch.device('cuda')
    model_config = transformers.RobertaConfig.from_pretrained(
        config.MODEL_CONFIG)
    model_config.output_hidden_states = True

    fold_models = []
    for i in range(config.N_FOLDS):
        model = models.TweetModel(conf=model_config)
        model.to(device)
        model.load_state_dict(torch.load(
            f'{config.TRAINED_MODEL_PATH}/model_{i}.bin'),
            strict=False)
        model.eval()
        fold_models.append(model)
    ## TweetDataset est un map-style et iterable-style datasets
    test_dataset = dataset.TweetDataset(
        tweets=df_test.text.values,
        sentiments=df_test.sentiment.values,
        selected_texts=df_test.selected_text.values)
    ## data_loader permet d'automatiser le chargement des données
    data_loader = torch.utils.data.DataLoader(
        test_dataset,
        shuffle=False,
        batch_size=config.VALID_BATCH_SIZE,
        num_workers=4) ## shuffle=False puisuqe on est en mode evaluation donc pas besoin d'un chuffle

    char_pred_test_start = []
    char_pred_test_end = []
    ## Pas de calcu des gradiants, c'est un forward pass de notre modèle
    with torch.no_grad():
        tk0 = tqdm.tqdm(data_loader, total=len(data_loader))
        for bi, d in enumerate(tk0):
            ids = d['ids']
            token_type_ids = d['token_type_ids']
            mask = d['mask']
            orig_tweet = d['orig_tweet']
            offsets = d['offsets']

            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)

            outputs_start_folds = []
            outputs_end_folds = []
            for i in range(config.N_FOLDS):
                outputs_start, outputs_end = \
                    fold_models[i](ids=ids,
                                   mask=mask,
                                   token_type_ids=token_type_ids)
                outputs_start_folds.append(outputs_start)
                outputs_end_folds.append(outputs_end)

            outputs_start = sum(outputs_start_folds) / config.N_FOLDS
            outputs_end = sum(outputs_end_folds) / config.N_FOLDS

            outputs_start = torch.softmax(outputs_start, dim=-1).cpu().detach().numpy()
            outputs_end = torch.softmax(outputs_end, dim=-1).cpu().detach().numpy()
            ## Affecter les prababilitées de l'output  outputs_start/outputs_end au char
            ## pour passer au char level puisque le Transformers sont token level
            ## chaque caractére prends la probavilitée affecté au token auquel il appartient
            for px, tweet in enumerate(orig_tweet):
                char_pred_test_start.append(
                    utils.token_level_to_char_level(tweet, offsets[px], outputs_start[px]))
                char_pred_test_end.append(
                    utils.token_level_to_char_level(tweet, offsets[px], outputs_end[px]))
    ## Serialiser et sauvegarder les output de la prédiction
    with open('/content/drive/MyDrive/very_final/pickles/roberta-char_pred_test_start.pkl', 'wb') as handle:
        pickle.dump(char_pred_test_start, handle)
    with open('/content/drive/MyDrive/very_final/pickles/roberta-char_pred_test_end.pkl', 'wb') as handle:
        pickle.dump(char_pred_test_end, handle)


if __name__ == '__main__':
    run()

100%|██████████| 111/111 [01:02<00:00,  1.78it/s]


# models.py

---
ce fichier contient une implémentation de la classe de modèle **TweetModel** qui héritée des transformateurs **BertPreTrainedModel** et la méthode forward qui récupère la sortie logits juste avant la couche des embeddings et effectue un Max-pooling et un Average_pooling


In [ ]:
import torch
import transformers

import config


class TweetModel(transformers.BertPreTrainedModel):
    ## Instantaition du modele
    def __init__(self, conf):
        super(TweetModel, self).__init__(conf)
        self.roberta = transformers.RobertaModel.from_pretrained(
            config.MODEL_CONFIG,
            config=conf)
        self.high_dropout = torch.nn.Dropout(config.HIGH_DROPOUT)
        self.classifier = torch.nn.Linear(config.HIDDEN_SIZE * 2, 2)

        torch.nn.init.normal_(self.classifier.weight, std=0.02)

    def forward(self, ids, mask, token_type_ids):
        # sequence_output of N_LAST_HIDDEN + Embedding states
        # (N_LAST_HIDDEN + 1, batch_size, num_tokens, 768)
        _, _, out = self.roberta(ids, attention_mask=mask,
                                 token_type_ids=token_type_ids)
        
        ## Récupére les valeus de toutes les couches sans la couche des embeddings.
        out = torch.stack(
            tuple(out[-i - 1] for i in range(config.N_LAST_HIDDEN)), dim=0)
        ## Avg pooling
        out_mean = torch.mean(out, dim=0)
        ## Max pooling
        out_max, _ = torch.max(out, dim=0)
        out = torch.cat((out_mean, out_max), dim=-1)


        # Multisample Dropout: https://arxiv.org/abs/1905.09788 expliqué just en bas
        ## logit céest la couche qui vient just avant la couche Dense
        logits = torch.mean(torch.stack([
            self.classifier(self.high_dropout(out))
            for _ in range(5)
        ], dim=0), dim=0)
        ## puique on'a deux output dans la loits (start_lable / end_label)
        start_logits, end_logits = logits.split(1, dim=-1)

        # (batch_size, num_tokens)
        ## .squeeze() pou applatire (flatteniser) les Nd tensors
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        return start_logits, end_logits

**Multi Sample Dropout (MSD):** C'est l'une des techniques qu'ils ont utilisées et que je trouve si intéressante. En fait, il applique un dropout plusieurs fois avec différents masques et ensuite il calcule la moyenne des résultats</br>
  Le dropout initial crée un sous-ensemble choisi au hasard (appelé dropout sample) à partir des données d'entrée de chaque itération d'entrainement, tandis que le MSD crée plusieurs échantillon de dropout. La loss est calculée pour chaque échantillon, puis la moyenne des losses des échantillons est calculée pour obtenir la Loss finale [(plus de détails ici)](https://arxiv.org/pdf/1905.09788.pdf).</br>
  ![alt text](https://github.com/Amine-OMRI/tweet-sentiment-extraction-kaggle-compete-1st-place-detailed-solution/blob/main/Multi-Sample-Dropout.png?raw=true)</br>

# utils.py

---

le fichier **utils.py**contient toutes les implémentations de toutes les fonctions qui seront utilisées dans de nombreux fichiers de code, telle que :
* la fonction qui fixe le seed global **seed_everything** 
la fonction qui calcule les probabilités de niveau de caractères token_level_to_char_level
* la fonction qui calcule la métrique de l'évaluation mentionnée dans les énoncées de la competition, qui est **jaccard**
* la fonction qui calcule le score final du Jaccard en utilisant les prédictions **calculate_jaccard**

In [ ]:
import os
import random

import torch
import numpy as np


def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)


def token_level_to_char_level(text, offsets, preds):
    probas_char = np.zeros(len(text))
    for i, offset in enumerate(offsets):
        if offset[0] or offset[1]:
            probas_char[offset[0]:offset[1]] = preds[i]

    return probas_char


def jaccard(str1, str2):
    """Original metric implementation."""
    a = set(str1.lower().split())
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


def get_best_start_end_idx(start_logits, end_logits,
                           orig_start, orig_end):
    """Return best start and end indices following BERT paper."""
    best_logit = -np.inf
    best_idxs = None
    start_logits = start_logits[orig_start:orig_end + 1]
    end_logits = end_logits[orig_start:orig_end + 1]
    for start_idx, start_logit in enumerate(start_logits):
        for end_idx, end_logit in enumerate(end_logits[start_idx:]):
            logit_sum = start_logit + end_logit
            if logit_sum > best_logit:
                best_logit = logit_sum
                best_idxs = (orig_start + start_idx,
                             orig_start + start_idx + end_idx)
    return best_idxs


def calculate_jaccard(original_tweet, target_string,
                      start_logits, end_logits,
                      orig_start, orig_end,
                      offsets, 
                      verbose=False):
    """Calculates final Jaccard score using predictions."""
    start_idx, end_idx = get_best_start_end_idx(
        start_logits, end_logits, orig_start, orig_end)

    filtered_output = ''
    for ix in range(start_idx, end_idx + 1):
        filtered_output += original_tweet[offsets[ix][0]:offsets[ix][1]]
        if (ix + 1) < len(offsets) and offsets[ix][1] < offsets[ix + 1][0]:
            filtered_output += ' '

    # Return orig tweet if it has less then 2 words
    if len(original_tweet.split()) < 2:
        filtered_output = original_tweet

    if len(filtered_output.split()) == 1:
        filtered_output = filtered_output.replace('!!!!', '!')
        filtered_output = filtered_output.replace('..', '.')
        filtered_output = filtered_output.replace('...', '.')

    filtered_output = filtered_output.replace('ïï', 'ï')
    filtered_output = filtered_output.replace('¿¿', '¿')

    jac = jaccard(target_string.strip(), filtered_output.strip())
    return jac, filtered_output


class AverageMeter:
    """Computes and stores the average and current value."""
    ## Permet de stocke la valeur moyenne actuelle et applique deux méthodes :
    ## reset : qui remet toutes les valeurs à zéro 
    ## update : qui met à jour l'objet en y ajoutant de nouvelles valeurs, ici il s'agit de la valeur de la loss
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

#train.py

---
Ce fichier de code exécute l'entrainement sur les données d'entranement et de la validation sur les données de validation et affiche les valeurs de jaccard et la loss

1) Il a utilisé le GPU Colab Pro pour RoBERTa-large et il a fallu environ 6h pour l'entraîner avec 5 folds et 4 époques sans optimisation particulière. 

[Hikkiiii](https://www.kaggle.com/wochidadonggua) a également entraîner RoBERTa-large, 2V100, APEX(O1), il a fallu environ 220s par époque 

2) RoBERTa-base-squad2 est disponible pré-entrainé par [HuggingFace.](https://huggingface.co/deepset/roberta-base-squad2)


In [ ]:
import numpy as np
import pandas as pd
import transformers
import torch
import torchcontrib

import config
import dataset
import models
import engine
import utils

Une autre technique qui a été utilisée comme **Optimiser** l'est :
* **SWA :** la technique SWA [(Stochastic Weight Averaging)](https://pytorch.org/blog/stochastic-weight-averaging-in-pytorch/) récemment proposée, et sa nouvelle implémentation dans torchcontrib. La SWA est une procédure simple qui améliore la généralisation du deep learning sur la Descente de Gradient Stochastique (SGD) sans coût supplémentaire, et peut être utilisée en remplacement de tout autre **optimiseur dans PyTorch**. Le SWA a une large gamme d'applications et de fonctionnalités.</br>
Il a été démontré que SWA améliore considérablement la généralisation des tâches de vision par ordinateur, y compris les VGG, les ResNets, les Wide ResNets et les DenseNets sur ImageNet et les CIFAR benchmarks.

En bref, le SWA effectue une moyenne égale des poids traversés par le SGD avec un programme d'apprentissage modifié.

In [ ]:
## Ce code permet de lancé l'entrainement sur les 5 folds 
def run(fold):
   
    dfx = pd.read_csv(config.TRAINING_FILE)
    df_train = dfx[dfx.kfold != fold].reset_index(drop=True)
    df_valid = dfx[dfx.kfold == fold].reset_index(drop=True)

    train_dataset = dataset.TweetDataset(
        tweets=df_train.text.values,
        sentiments=df_train.sentiment.values,
        selected_texts=df_train.selected_text.values)

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.TRAIN_BATCH_SIZE,
        num_workers=4,
        shuffle=True)

    valid_dataset = dataset.TweetDataset(
        tweets=df_valid.text.values,
        sentiments=df_valid.sentiment.values,
        selected_texts=df_valid.selected_text.values)

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=config.VALID_BATCH_SIZE,
        num_workers=4,
        shuffle=False)

    device = torch.device('cuda')
    model_config = transformers.RobertaConfig.from_pretrained(
        config.MODEL_CONFIG)
    model_config.output_hidden_states = True
    model = models.TweetModel(conf=model_config)
    model = model.to(device)
    print("------------------> here")
    
    ## Nombre d'iteration c'est le nombre des données d'entré (tweets) divisé par
    ## la taille du batch dans le fichier config.py
    num_train_steps = int(
        len(df_train) / config.TRAIN_BATCH_SIZE * config.EPOCHS)
    ## Récupérer les paramètres de l'optimizer
    param_optimizer = list(model.named_parameters())

    ## puisqu'il est recommandé d'utiliser cet optimiseur pour le fine tuning
    ## (modification sur l'architecture), puisque c'est ainsi que le modèle a 
    ## été entraîné et de conserver les mêmes comportements que ceux mentionnés 
    ## sur le repo git de BERT (https://github.com/google-research/bert/blob/master/optimization.py#L65)
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_parameters = [
        {'params': [p for n, p in param_optimizer
                    if not any(nd in n for nd in no_decay)],
         'weight_decay': config.WEIGHT_DECAY},
        {'params': [p for n, p in param_optimizer
                    if any(nd in n for nd in no_decay)],
         'weight_decay': 0.0}]
    ## AdamW: un optimiseur adaptatif avec utilisation d'une échelle de taux
    ## d'apprentissage pour moduler l'évolution du taux d'apprentissage de
    ## l'optimiseur en fonction du temps 
    base_opt = transformers.AdamW(optimizer_parameters,
                                  lr=config.LEARNING_RATE)
    ## SWA : la technique SWA (Stochastic Weight Averaging) est présenter au dessus de cette cellule
    optimizer = torchcontrib.optim.SWA(
        base_opt,
        swa_start=int(num_train_steps * config.SWA_RATIO),
        swa_freq=config.SWA_FREQ,
        swa_lr=None)
    
    scheduler = transformers.get_linear_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=int(num_train_steps * config.WARMUP_RATIO),
        num_training_steps=num_train_steps)

    print(f'Training is starting for fold={fold}')

    for epoch in range(config.EPOCHS):
        engine.train_fn(train_data_loader, model, optimizer,device, scheduler=scheduler)
        jaccard = engine.eval_fn(valid_data_loader, model, device)

    if config.USE_SWA:
        optimizer.swap_swa_sgd()

    torch.save(model.state_dict(),
               f'{config.MODEL_SAVE_PATH}/model_{fold}.bin')

    return jaccard


if __name__ == '__main__':
    utils.seed_everything(seed=config.SEED)

    fold_scores = []
    for i in range(config.N_FOLDS):
        fold_score = run(i)
        fold_scores.append(fold_score)

    print('\nScores without SWA:')
    for i in range(config.N_FOLDS):
        print(f'Fold={i}, Jaccard = {fold_scores[i]}')
    print(f'Mean = {np.mean(fold_scores)}')
    print(f'Std = {np.std(fold_scores)}')

  0%|          | 0/687 [00:00<?, ?it/s]

------------------> here
Training is starting for fold=0


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2398: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."
/usr/local/lib/python3.6/dist-packages/transformers/optimization.py:155: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)
  0%|          | 0/687 [00:00<?, ?it/s]

Jaccard = 0.7029905612369929


  0%|          | 0/687 [00:00<?, ?it/s]

...

Jaccard = 0.70296628740658


  0%|          | 0/687 [00:00<?, ?it/s]

Jaccard = 0.7067339098954668


100%|██████████| 172/172 [00:34<00:00,  4.94it/s, jaccard=0.711, loss=0.00943]


Jaccard = 0.7109265704332721

Scores without SWA:
Fold=0, Jaccard = 0.7163026420895469
Fold=1, Jaccard = 0.7108215431021135
Fold=2, Jaccard = 0.7158955035127735
Fold=3, Jaccard = 0.7166221978997807
Fold=4, Jaccard = 0.7109265704332721
Mean = 0.7141136914074974
Std = 0.002655369837057321
